In [3]:
import random

## NN Structure

output policy will have a dimension of the sum of discard choices, play choices, and clue choices. clue choices has a cardinality of product of num players, num colors, and num numbers. In this case it will be 4+4+3*3*1= 17 element array

Input layer will have a dimension of the sum of opp_hand, board, discard, self_clue, and opp_clue.
opp_hand will be dummy encoding of the number of cards in the deck. board is the number of playable cards. self_clue is product of num of cards_in_hand, num of colors, and num of value, so 4*3*3 = 36. Same with opp_clue

Input layer = 15+9+15+36+36 = 111 node
also add in num bombs, number of cards left, number of clues, 4, 15-4*2=7, 8

total input layer = 111+4+7+8 = 130


In [4]:
class Card:
    def __init__(self, value, color):
        self.value = value
        self.color = color
        
    def __str__(self):
        return(str(self.value)+self.color)

class Deck:
    def __init__(self, cards):
        self.cards = cards
        
    def __str__(self):
        return(str([str(c) for c in self.cards]))
        
    def count(self):
        return len(self.cards)
    
    def shuffle(self):
        random.shuffle(cards)
    
    def deal(self,n):
        if self.count()>=n:
            dealt = self.cards[0:n]
            self.cards = self.cards[n:]
            return (dealt)
    
      

In [73]:
class info:
    def __init__(self):
        self.value = 0
        self.not_value = []
        self.color = "NA"
        self.not_color = []
        self.playable_prob = 0
        self.discardable_prob = 0
    
#     def __str__(self):
#         return()

class Player:       
    def __init__(self, name, deck):
        self.name = name
        self.hand = []
        self.info = [info(), info(), info(), info()]
        self.seen = []
        self.remaining = deck.cards.copy()
        
    def __str__(self):
        return(str([str(c) for c in self.hand]))
    
    def play_card(self, pos):
        played = self.hand[pos]
        print("%s:"%self.name,self," played card num %d: %s"%(pos,played))
        del self.hand[pos]
        return played
        
    def draw(self, cards):
        for c in cards:
            self.hand.append(c)
    
    def see(self, playable, discardable, players = [], played = []):
        for p in players:
            self.seen += p.hand
        self.seen+=played
        self.seen = list(set(self.seen))
        self.remaining = [x for x in self.remaining if x not in self.seen]            
        print("%s has seen:"%(self.name),[x.__str__() for x in self.seen])
        
        for i in self.info:
            possible = self.remaining.copy()
            playable = playable.copy()
            discardable = discardable.copy()
            if i.value>0:
                possible = [x for x in possible if x.value==i.value]
            if i.color!="NA":
                possible = [x for x in possible if x.color==i.color]
            
            d = [x for x in possible if x.__str__() in [di.__str__() for di in discardable]]
            p = [x for x in possible if x in playable]
            
            i.playable_prob = len(p)/len(possible)
            i.discardable_prob = len(d)/len(possible)

#             print("playable",[x.__str__() for x in p],len(p))
#             print("discard",[x.__str__() for x in d],len(d))
#             print("possible",[x.__str__() for x in possible], len(possible))
    
    def receive_clue(self,other,clue):
        if type(clue)==str:
            idx = [i for i, x in enumerate(self.hand) if x.color == clue]
            if not idx == []:
                for i in range(0,4):
                    if i in idx:
                        self.info[i].color=clue
                    else:
                        self.info[i].not_color.append(clue)
                        
        if type(clue)==int:
            idx = [i for i, x in enumerate(self.hand) if x.value == clue]
            if not idx == []:
                for i in range(0,4):
                    if i in idx:
                        self.info[i].value=clue
                    else:
                        self.info[i].not_value.append(clue)
                        
        print("%s gave clue: %s to %s"%(other.name,str(clue),self.name))

In [74]:
class Board:
    def __init__(self, colors, cards):
        d = dict.fromkeys(colors)
        d = dict.fromkeys(d, [0])
        self.slots = d
        self.deck = cards.copy()
        self.bombs = 0
        self.clues = 8
        self.score = 0
        self.played = []
        self.playable = [x for x in self.deck if x.value==1]
        self.discardable = []
        
    def play_card(self, card):
        self.played.append(card)
        slot = self.slots[card.color]
        last = slot[-1]
        if card.value == last+1:
            self.slots[card.color] = slot +[card.value]
            self.score +=1
            print(card, " was played correctly")
            self.discardable.append(card)
            self.playable = [x for x in self.playable if(x.value!=card.value) or (x.color!=card.color)]
            self.playable+=[x for x in self.deck if (x.value==(card.value+1)) and (x.color==card.color)]
        else:
            self.bombs += 1
            print(card, " was NOT played correctly")
        
        self.played += [card]

    

In [75]:
cards = [Card(value, color) for color in ["R","B","Y"] for value in [1,1,2,2,3]]
deck = Deck(cards = cards)
board = Board(["R","B","Y"], cards)
p1 = Player("P1", cards)
p2 = Player("P2", cards)

deck.shuffle()
p1.draw(deck.deal(4))
p1.hand

[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]

In [76]:
p1._print()

['1R', '3R', '1Y', '3Y']


In [56]:
a = [1,2,3,1]
b = [5,3,4,1,2]

a.index(1)

0

In [37]:
p1.receive_clue(p2,1)

P2 gave clue: 1 to P1


In [38]:
for i in p1.info:
    print(i.value, i.not_value, i.color, i.not_color)

1 [] NA []
0 [1] NA []
1 [] NA []
1 [] NA []


In [480]:
players = [p1,p2]
turn_num = 0
while deck.count()>0:
    turn = turn_num%2
    print("turn %d"%turn_num)
    print(p1,p2)
    turn_num+=1
    curr_player = players[turn]
    other_players = [x for x in players if x != curr_player]
    curr_player.see(board.playable, board.discardable, players = other_players, played = board.played)
    r = random.randint(0,3)
    played = curr_player.play_card(r)
    curr_player.draw(deck.deal(1))
    board.play_card(played)
    print([x.__str__() for x in board.playable])

#last round
for turn in range(0,2):
    turn_num+=1
    print("turn %d"%turn_num)
    curr_player = players[turn]
    other_players = [x for x in players if x != curr_player]
    curr_player.see(board.playable, board.discardable, players = other_players, played = board.played)
    r = random.randint(0,3)
    played = curr_player.play_card(r)
    board.play_card(played)
print(board.slots)

turn 0
['2Y', '1Y', '1R', '3Y'] ['2B', '1B', '3B', '3R']
P1 has seen: ['3B', '3R', '2B', '1B']
P1: ['2Y', '1Y', '1R', '3Y']  played card num 1: 1Y
1Y  was played correctly
['1R', '1B', '1R', '1B', '2Y', '2Y']
turn 1
['2Y', '1R', '3Y', '2R'] ['2B', '1B', '3B', '3R']
P2 has seen: ['2Y', '3Y', '1Y', '1R', '2R']
P2: ['2B', '1B', '3B', '3R']  played card num 1: 1B
1B  was played correctly
['1R', '1R', '2Y', '2Y', '2B', '2B']
turn 2
['2Y', '1R', '3Y', '2R'] ['2B', '3B', '3R', '1R']
P1 has seen: ['1Y', '1B', '1R', '3R', '2B', '3B']
P1: ['2Y', '1R', '3Y', '2R']  played card num 0: 2Y
2Y  was played correctly
['1R', '1R', '2B', '2B', '3Y']
turn 3
['1R', '3Y', '2R', '2B'] ['2B', '3B', '3R', '1R']
P2 has seen: ['2Y', '3Y', '1Y', '1B', '2B', '1R', '2R']
P2: ['2B', '3B', '3R', '1R']  played card num 1: 3B
3B  was NOT played correctly
['1R', '1R', '2B', '2B', '3Y']
turn 4
['1R', '3Y', '2R', '2B'] ['2B', '3R', '1R', '1Y']
P1 has seen: ['2Y', '1Y', '1B', '1R', '3R', '2B', '3B', '1Y']
P1: ['1R', '3Y', 

In [349]:
cards = [Card(value, color) for value in [1,1,2,2,3] for color in ["R","B","Y"]]
deck = Deck(cards = cards)
deck.shuffle()
deck.cards

In [339]:
seen = deck.cards[0:3]
seen

In [351]:
[x for x in deck.cards if x.value == 1]

TypeError: 'str' object is not callable